In [1]:
# Importing Libraries 
import pandas as pd 
import sys 
from pathlib import Path
import json
sys.path.append(str(Path.cwd().parents[0]))

from src.train import (training_model, dumm_model, fit_and_pred, stat_metrics, log_pipeline, GB_pipe, RF_pipe)

In [2]:
# Retrieving columns used 
cat_cols = json.load(open('categorical_cols.json'))
num_cols = json.load(open('numerical_cols.json'))
features = json.load(open('feature_list.json'))

In [11]:
# Recreating the split from previous file 
X_train, X_test, y_train, y_test = training_model()

# Dummy pipeline model
dummy_pipeline = dumm_model(cat_cols, num_cols)

# fitting an predicting values 
y_pred, y_prob = fit_and_pred(dummy_pipeline, X_train, y_train, X_test)

# getting important testing metrics 
roc, precision, recall = stat_metrics(y_test, y_pred, y_prob)


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [12]:
results = {
    "model": "Dummy (most frequent)",
    "roc_auc": roc,
    "recall": recall,
    "precision": precision}

results

{'model': 'Dummy (most frequent)',
 'roc_auc': 0.5,
 'recall': 0.0,
 'precision': 0.0}

In [15]:
# Training and splitting the model 
LR_X_train, LR_X_test, LR_y_train, LR_y_test = training_model()

# Logistic Regression pipeline model
LR_pipeline = log_pipeline(cat_cols, num_cols)

# fitting an predicting values 
LR_y_pred, LR_y_prob = fit_and_pred(LR_pipeline, LR_X_train, LR_y_train, LR_X_test)

# getting important testing metrics 
LR_roc, LR_precision, LR_recall = stat_metrics(LR_y_test, LR_y_pred, LR_y_prob)

In [16]:
LR_results = {
    "model": "Logistic Regression (balanced)",
    "roc_auc": LR_roc,
    "recall": LR_recall,
    "precision": LR_precision}

LR_results

{'model': 'Logistic Regression (balanced)',
 'roc_auc': 0.6422622696318383,
 'recall': 0.5486569793042713,
 'precision': 0.1669793621013133}

In [7]:
# Training and splitting the model 
RF_X_train, RF_X_test, RF_y_train, RF_y_test = training_model()

# Logistic Regression pipeline model
RF_pipeline = RF_pipe(cat_cols, num_cols)

# fitting an predicting values 
RF_y_pred, RF_y_prob = fit_and_pred(RF_pipeline, RF_X_train, RF_y_train, RF_X_test)

# getting important testing metrics 
RF_roc, RF_precision, RF_recall = stat_metrics(RF_y_test, RF_y_pred, RF_y_prob)

RF_results = {
    "model": "Random Forest",
    "roc_auc": RF_roc,
    "recall": RF_recall,
    "precision": RF_precision
}

In [8]:
# Training and splitting the model 
GB_X_train, GB_X_test, GB_y_train, GB_y_test = training_model()

# Logistic Regression pipeline model
GB_pipeline = GB_pipe(cat_cols, num_cols)

# fitting an predicting values 
GB_y_pred, GB_y_prob = fit_and_pred(GB_pipeline, GB_X_train, GB_y_train, GB_X_test)

# getting important testing metrics 
GB_roc, GB_precision, GB_recall = stat_metrics(GB_y_test, GB_y_pred, GB_y_prob)

GB_results = {
    "model": "Gradient Boosting",
    "roc_auc": GB_roc,
    "recall": GB_recall,
    "precision": GB_precision
}

In [17]:
result_df = pd.DataFrame([results, LR_results, RF_results, GB_results])
result_df



,model,roc_auc,recall,precision
0,Dummy (most frequent),0.500000,0.000000,0.000000
1,Logistic Regression (balanced),0.642262,0.548657,0.166979
2,Random Forest,0.661690,0.559665,0.173919
3,Gradient Boosting,0.677373,0.008807,0.625000


Logistic regression model outperforms the Dummy classifier meaning that the model was able to learn and perform better. 